In [ ]:
import itertools
import os
import pathlib
from os import path
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

class_name = re.compile('mw-category.*')
href_name=re.compile('.*?\.jpg',re.IGNORECASE)
folder = r"C:\Users\User\Desktop\automobiles"

In [ ]:
def get_links(url):
    links = []
    names = []
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    categories_list = soup.find(class_=class_name)
    categories = categories_list.find_all('a', href=True)
    for category in categories:
        links.append('https://it.wikipedia.org/' + category.get('href'))
        names.append(category.get('title'))
    return dict(zip(names, links))

In [ ]:
array_categories = get_links('https://it.wikipedia.org/wiki/Categoria:Automobili_per_marca')
for k, v in dict(itertools.islice(array_categories.items(), 20)).items():
    sub_elems = get_links(v)
    for m, n in dict(itertools.islice(sub_elems.items(), 20)).items():
        if 'Categoria' not in n:
            response = requests.get(n)
            soup = BeautifulSoup(response.text, 'html.parser')
            image = soup.find("a", href=href_name)

            first_level=os.path.join(folder, k.replace(":", " "),re.sub("[:\/\"]"," ",m))
            os.makedirs(first_level,exist_ok=True)
            url = "https:" +  image.find("img")['src']
            Picture_request = requests.get(url)
            if Picture_request.status_code == 200:
                with open(os.path.join(first_level,re.sub("[:\/\"]"," ",m)+".jpg"),"wb")as f:
                    f.write(Picture_request.content)